In [1]:
from manage_model import create_model, allowgrowthgpus, fit_model, mm_load_model
from best_models import retrieve_best_models, retrieve_models_by_id
from check_and_prepare_dataset import load_dataset, prepare_dataset
import pandas as pd

2023-07-07 16:20:59.776528: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
filters = ["df_flt['params'].str.contains(\"'dropout': None\")"] # no dropout
#filters = ["~df_flt['params'].str.contains(\"'dropout': None\")"] # with dropout
opt_targets = ['auc', 'f1-score 1', 'hybrid1', 'hybrid2', 'hybrid5', 'NH2', 'NH5', 'NH10']
resdir = '/home/lstam/Documents/bestmodels'
testfpattern=cvrespattern = '*NN_ns*mean*'
#testmodel = retrieve_models_by_id(197,'auc test', resdir, testfpattern, opt_targets, 'val.', 'test', filters, 3)

#testmodel = retrieve_models_by_id(875,'NH2 test', resdir, testfpattern, opt_targets, 'val.', 'test', filters, 3)

testmodel = retrieve_models_by_id(785,'hybrid2 test', resdir, testfpattern, opt_targets, 'val.', 'test', filters, 3)




In [3]:
testmodel

{'params': {'ES_mindelta': 0.002,
  'ES_monitor': 'val_loss',
  'ES_patience': 10,
  'batch_size': 512,
  'class_weights': {0: 1, 1: 5},
  'dropout': None,
  'feature_drop': ('month',
   'weekday',
   'dom_dir',
   'dir_max',
   'pop',
   'corine_gr1',
   'corine_gr2',
   'corine_gr3',
   'corine_gr4',
   'corine_gr5',
   'corine_gr6',
   'corine_gr7',
   'corine_gr8',
   'corine_gr9'),
  'max_epochs': 2000,
  'metric': 'accuracy',
  'n_internal_layers': (0, {'layer_1_0_nodes': 1300.0}),
  'optimizer': {'adam_params': None, 'name': 'Adam'}},
 'trial': 785}

In [5]:
tset='/home/lstam/Documents/trainsets/train/train_new_sample_1_2_norm.csv'
Xt, yt, gt=load_dataset(tset, testmodel['params']['feature_drop'])

Loading full dataset /home/lstam/Documents/trainsets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_month_6', 'bin_month_9', 'bin_dir_max_8', 'bin_weekday_5', 'bin_dom_dir_2', 'bin_weekday_3', 'bin_dir_max_6', 'bin_corine_gr22', 'bin_dom_dir_7', 'bin_corine_gr23', 'bin_dir_max_1', 'bin_dir_max_4', 'bin_corine_gr31', 'bin_weekday_2', 'bin_dom_dir_4', 'bin_corine_gr32', 'bin_dir_max_2', 'bin_corine_gr1', 'bin_dir_max_7', 'bin_dom_dir_5', 'bin_corine_gr5', 'bin_corine_gr21', 'bin_dom_dir_8', 'bin_weekday_4', 'bin_dom_dir_1', 'bin_month_3', 'bin_month_4', 'bin_weekday_6', 'bin_month_8', 'bin_corine_gr33', 'bin_month_7', 'bin_dir_max_3', 'bin_weekday_1', 'pop', 'bin_month_10', 'bin_month_5', 'bin_dir_max_5', 'bin_weekday_7', 'bin_corine_gr4', 'bin_dom_dir_3', 'bin_do

In [4]:
dsetf='/home/lstam/Documents/daily_rasters/csv/20230702_norm.csv'
#dsetf='/home/lstam/Documents/trainsets/20190803_df_norm.csv'

In [5]:
X_pd, y_pd, g=load_dataset(dsetf, testmodel['params']['feature_drop'])
X_pd = X_pd.reindex(sorted(X_pd.columns), axis=1)
X = X_pd.values
y = y_pd.values
y = y[:, 0]

Loading full dataset /home/lstam/Documents/daily_rasters/csv/20230702_norm.csv
before nan drop: 654111
after nan drop: 654111
after dup. drop: 654111
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['x', 'y', 'dom_dir', 'dir_max', 'firedate', 'band', 'crs', 'weekday', 'month', 'fire', 'id']
Dropped columns ['bin_month_5', 'bin_weekday_3', 'bin_dom_dir_3', 'bin_weekday_2', 'bin_month_9', 'bin_dom_dir_8', 'bin_month_4', 'bin_dir_max_2', 'bin_corine_gr31', 'bin_month_7', 'bin_dir_max_8', 'bin_dir_max_1', 'bin_corine_gr24', 'bin_corine_gr5', 'bin_dom_dir_5', 'bin_month_8', 'bin_dir_max_5', 'bin_month_10', 'bin_corine_gr1', 'bin_weekday_7', 'bin_dom_dir_1', 'bin_weekday_4', 'bin_month_3', 'bin_corine_gr21', 'bin_dom_dir_4', 'bin_corine_gr4', 'bin_weekday_1', 'bin_dir_max_3', 'pop', 'bin_dir_max_6', 'bin_dom_dir_6', 'bin_corine_gr23', 'bin_corine_gr33', 'bin_weekday_5', 'bin_dom_dir_2', 'bin_dir_max_7', 'bin_corine_gr22', 'bin_weekday_6', 'bin_month_6', 'bin_dir_max_4', 'bin_dom_d

In [11]:
dset=pd.read_csv(dsetf)

In [6]:
#model=load_model('/home/lstam/Documents/bestmodels/models/hypres_tf_ns_ncv_do_2019_weights_id_197_r_0_auctest_1.cpkt', 'tf',\
#                 testmodel['params'],X)
#model=load_model('/home/lstam/Documents/bestmodels/models/hypres_tf_ns_ncv_do_2019_weights_id_240_r_0_hybrid2test_1.cpkt', 'tf',\
#                 testmodel['params'],X)
#model=load_model('/home/lstam/Documents/bestmodels/models/hypres_tf_ns_ncv_do_2019_weights_id_97_r_0_hybrid2test_1.cpkt', 'tf',\
#                 testmodel['params'],X)
mdl=None
mdl=mm_load_model('/home/lstam/Documents/bestmodels/entiremodels/hypres_tf_ns_ncv_do_2019_model_id_785_r_0_hybrid2test_1.h5')

#mdl=create_model('tf', testmodel['params'], X)



2023-07-07 16:21:18.213508: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [12]:
dset

,x,y,dom_dir,dom_vel,res_max,dir_max,max_temp,min_temp,mean_temp,max_dew_temp,...,corine_gr5,corine_gr21,corine_gr22,corine_gr23,corine_gr24,corine_gr31,corine_gr32,corine_gr33,fire,id
0,0.019069,0.767603,2.0,0.451685,0.451685,2.0,0.715023,0.826656,0.767943,0.844536,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,194770402300
1,0.019069,0.766878,2.0,0.451685,0.451685,2.0,0.715023,0.826656,0.767943,0.844536,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,194770402248
2,0.019069,0.766152,2.0,0.451685,0.451685,2.0,0.715023,0.826656,0.767943,0.844536,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,194770402197
3,0.019568,0.767603,2.0,0.451685,0.451685,2.0,0.715023,0.826656,0.767943,0.844536,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,194821402300
4,0.019568,0.766878,2.0,0.451685,0.451685,2.0,0.715023,0.826656,0.767943,0.844536,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,194821402248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654106,0.866827,0.225302,7.0,0.320371,0.320371,7.0,0.738300,0.855712,0.788669,0.888692,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0,282343363796
654107,0.867325,0.234013,7.0,0.263488,0.263488,7.0,0.743452,0.823020,0.785365,0.883634,...,0.572477,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,282395364415
654108,0.867325,0.226753,7.0,0.320371,0.320371,7.0,0.738300,0.855712,0.788669,0.888692,...,0.041635,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,282395363900
654109,0.867325,0.226028,7.0,0.320371,0.320371,7.0,0.738300,0.855712,0.788669,0.888692,...,0.091745,0.0,0.0,0.0,0.0,0.0,0.582454,0.0,0,282395363848


In [7]:
y=mdl.predict(X)

20441/20441 [==============================] - 44s 2ms/step


In [9]:
yp=y[:,0]
for i in range(0,5):
    print(i/5,len(yp[(yp>=i/5)&(yp<=i/5+0.2)]))

0.0 83416
0.2 58134
0.4 50373
0.6 62075
0.8 400113


In [14]:
out=pd.concat([dset['id'],pd.DataFrame(y, columns=['ypred0','ypred1'])], axis=1)

In [15]:
out

,id,ypred0,ypred1
0,194770402300,0.754613,0.245387
1,194770402248,0.762723,0.237277
2,194770402197,0.762575,0.237425
3,194821402300,0.718883,0.281117
4,194821402248,0.778790,0.221210
...,...,...,...
654106,282343363796,0.439716,0.560284
654107,282395364415,0.994298,0.005702
654108,282395363900,0.994477,0.005523
654109,282395363848,0.787247,0.212753


In [16]:
out['xu']=out['id'].astype(str).str.slice(0, 6).astype(int)/10000

In [17]:
out['yu']=out['id'].astype(str).str.slice(6, 12).astype(int)/10000

In [20]:
out['risk']=pd.np.digitize(out['ypred1'], bins = [0,0.2,0.4,0.6,0.8,1])

/tmp/ipykernel_18237/3092674785.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  out['risk']=pd.np.digitize(out['ypred1'], bins = [0,0.2,0.4,0.6,0.8,1])


In [21]:
out['risk'].unique()

array([2, 1, 3, 5, 4])